#### 1.1.2 _Programming Exercise:_ Verify that inverse nonce values `k` and `-k` generate inverse points `R` and `-R`

In [ ]:
# Generate a random scalar and its associated curve point
k, R = generate_key_pair()

# Find the x- and y-coordinates from R
# Use the get_x() and get_y() methods
R_x = R.get_x()
R_y = R.get_y()

print("R_x: {}".format(R_x))
print("R_y: {}\n".format(R_y))

# Find k's inverse (SECP256K1_ORDER - k)
# Extract the secret value from k using .secret
minus_k = SECP256K1_ORDER - k.secret

# Generate the key pair from minus_k using generate_key_pair() function with minus_k as an argument
minus_k_key, minus_R = generate_key_pair(minus_k)

# Find the x- and y-coordinates from -R
minus_R_x = minus_R.get_x()
minus_R_y = minus_R.get_y()

print("minus_R_x: {}".format(minus_R_x))
print("minus_R_y: {}\n".format(minus_R_y))

assert R_x == minus_R_x
assert SECP256K1_FIELD_SIZE - R_y == minus_R_y

print("Success!")

#### 1.1.3 _Programming Exercise:_ Sign a message with Schnorr

In [ ]:
msg = sha256(b'message')

# Generate a private/public key pair
d, P = generate_key_pair()

# Check that public key point has an even Y-coordinate.
# If not, negate d and P.
if P.get_y()%2 != 0:
    d.negate()
    P.negate()

# Generate a nonce scalar and associated nonce point
k, R = generate_key_pair()

# Check that nonce point has an even Y-coordinate.
# If not, negate k
if R.get_y()%2 != 0:
    k.negate()
# Note that there is no need to negate R, since we only use the x value of R below

# Generate s = k + hash(R_x|P_x|msg) * d
# Method: tagged_hash("BIP0340/challenge", bytes) will give you the byte digest tagged hash of the challenge data.
# Turn that digest into a ECKey object called h, and then set s = k + h * d
# Note that ECPubKey.get_bytes() will return the bip340 encoding of a public key which is equivalent 
# to its x-coordinate
R_x_bytes = R.get_bytes()
P_bytes = P.get_bytes()
h_bytes = tagged_hash("BIP0340/challenge", R_x_bytes + P_bytes + msg)
h = ECKey().set(h_bytes)
s = k + h * d

print("R: {}".format(R))
print("s: {}\n".format(s.get_bytes().hex()))

# Generate sig = R_x|s
# Method: get the x bytes from R and concatenate with the secret bytes from s
sig = R_x_bytes + s.get_bytes()

print("Signature: {}\n".format(sig.hex()))

# Verify the signature
assert P.verify_schnorr(sig, msg)

print("Success!")

#### 1.1.4 _Programming Exercise:_ Signing schnorr with deterministic nonce

In [ ]:
msg = sha256(b'message')
aux = sha256(b'random auxiliary data')

# Generate a valid BIP340 priv-pub key pair using the convenience function `generate_bip340_key_pair`
d, P = generate_bip340_key_pair()
print("message = {}".format(msg.hex()))
print("pubkey = {}\n".format(P.get_bytes().hex()))

# t is the byte-wise xor of bytes(d) and tagged_hash("BIP0340/aux", aux)
t = (d.secret ^ int.from_bytes(tagged_hash("BIP0340/aux", aux), 'big')).to_bytes(32, 'big')
rand = tagged_hash("BIP0340/nonce", t + P.get_bytes() + msg)

# Generate the nonce value k and get the nonce point R
k, R = generate_key_pair(rand)

# Check that nonce has an even y coordinate
# If not, negate k
if R.get_y()%2 != 0:
    k.negate()

print("nonce: {}".format(k))
print("nonce point: {}\n".format(R))

# Generate s = k + tagged_hash("BIP0340/challenge", R_x|P_x|msg) * d
# Method: tagged_hash("BIP0340/challenge", bytes) will give you the byte digest
# Turn that digest into a ECKey object called h, and then set s = k + h * d
R_x_bytes = R.get_bytes()
P_bytes = P.get_bytes()
h_bytes =  tagged_hash("BIP0340/challenge", R_x_bytes + P_bytes + msg)
h = ECKey().set(h_bytes)
s = k + h * d

print("R: {}".format(R))
print("s: {}\n".format(s.get_bytes()))

# Generate sig = R_x|s
# Method: get the x bytes from R and concatenate with the secret bytes from s
sig = R_x_bytes + s.get_bytes()

print("Signature: {}\n".format(sig.hex()))

# Generate a signature using the ECKey.sign_schnorr(msg, aux) method
# This generates the nonce deterministically, so should return the same signature
sig2 = d.sign_schnorr(msg, aux)

# Verify and compare signature(s)
assert P.verify_schnorr(sig, msg)
assert P.verify_schnorr(sig2, msg)
assert sig == sig2

print("Success!")